In [1]:
import sys 
sys.path.append('../scripts/')
from robot import *
from scipy.stats import multivariate_normal

In [2]:
class Particle:  ###impllikelihood
    def __init__(self, init_pose, weight):
        self.pose = init_pose
        self.weight = weight
        
    def func_state_transition(self, nu, omega, time, pose): 
        t0 = pose[2]
        if math.fabs(omega) < 1e-10:
            return pose + np.array( [nu*math.cos(t0), 
                                     nu*math.sin(t0),
                                     omega ] ) * time
        else:
            return pose + np.array( [nu/omega*(math.sin(t0 + omega*time) - math.sin(t0)), 
                                     nu/omega*(-math.cos(t0 + omega*time) + math.cos(t0)),
                                     omega*time ] )
        
    def motion_update(self, nu, omega, time, noise_rate_pdf): 
        ns = noise_rate_pdf.rvs()
        pnu = nu + ns[0]*math.sqrt(nu/time) + ns[1]*math.sqrt(omega/time)
        pomega = omega + ns[2]*math.sqrt(nu/time) + ns[3]*math.sqrt(omega/time)
        self.pose = self.func_state_transition(pnu, pomega, time, self.pose)
        
    def observation_update(self, observation, envmap):  #変更
        for d in observation:
            lid = d[1]
            actual = envmap.landmarks[lid]
            print(actual.pos)            
        

In [3]:
class Mcl:  ###recvmapmcl
    def __init__(self, envmap, init_pose, num, motion_noise_vars):    #envmapを追加 
        self.particles = [Particle(init_pose, 1.0/num) for i in range(num)]
        self.map = envmap  #追加

        v = motion_noise_vars
        c = np.diag([v["nn"]**2, v["no"]**2, v["on"]**2, v["oo"]**2])
        self.motion_noise_rate_pdf = multivariate_normal(cov=c)
        
    def motion_update(self, nu, omega, time):
        for p in self.particles: p.motion_update(nu, omega, time, self.motion_noise_rate_pdf)
            
    def observation_update(self, observation):
        for p in self.particles: p.observation_update(observation, self.map) #引数追加
        
    def draw(self, ax, elems):
        xs = [p.pose[0] for p in self.particles]
        ys = [p.pose[1] for p in self.particles]
        vxs = [math.cos(p.pose[2]) for p in self.particles]
        vys = [math.sin(p.pose[2]) for p in self.particles] 
        elems.append(ax.quiver(xs, ys, vxs, vys, color="blue", alpha=0.5))

In [4]:
class MclAgent(Agent):  ###recvmap (1-6行目)
    def __init__(self, time_interval, nu, omega, particle_pose, envmap, particle_num=100, \
                motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2}): #2行目にenvmapを追加
        super().__init__(nu, omega)
        self.mcl = Mcl(envmap, particle_pose, particle_num, motion_noise_stds) #envmapを追加
        self.time_interval = time_interval
        
    def decision(self, observation=None):
        self.mcl.motion_update(self.nu, self.omega, self.time_interval)
        self.mcl.observation_update(observation)
        return self.nu, self.omega
        
    def draw(self, ax, elems):
        self.mcl.draw(ax, elems)

In [5]:
if __name__ == '__main__':          ###addmap （13行目のみ）
    time_interval = 0.1
    world = World(30, time_interval) 

    ### 地図を生成して3つランドマークを追加 ###
    m = Map()                                  
    m.append_landmark(Landmark(-4,2))
    m.append_landmark(Landmark(2,-3))
    m.append_landmark(Landmark(3,3))
    world.append(m)          

    ### ロボットを作る ###
    circling = MclAgent(time_interval, 0.2, 10.0/180*math.pi, np.array([0, 0, 0]).T, m) #地図を引数で渡す
    r = Robot(np.array([0,0,0]).T, sensor=Camera(m), agent=circling, color="red")
    world.append(r)

    #world.draw()                       # アニメーションさせるとき
    r.one_step(time_interval)  # アニメーションなしでデバッグするとき

[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
[ 2 -3]
